# Resources
[CheXNet: Radiologist-Level Pneumonia Detection on Chest X-Rays with Deep Learning](https://stanfordmlgroup.github.io/projects/chexnet/)

[Can Machine Learning Read Chest X-rays like Radiologists?](https://towardsdatascience.com/can-machine-learning-read-chest-x-rays-like-radiologists-part-1-7182cf4b87ff)

[Efcient Deep Network Architectures for Fast Chest X-Ray Tuberculosis Screening and Visualization](https://www.nature.com/articles/s41598-019-42557-4.pdf)

# Medical Torch
https://github.com/perone/medicaltorch

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Check image size

In [ ]:
path = '../input/chest_xray/chest_xray/train/PNEUMONIA'
name = os.listdir(path)[1]
img = Image.open(path+"/"+name)
img.size

# Compare two image

In [ ]:
from PIL import Image
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(55,45))
path = '../input/chest_xray/chest_xray/train/PNEUMONIA'
name = os.listdir(path)[1]
img = Image.open(path+"/"+name)
ax = fig.add_subplot(1, 5,  1, xticks=[], yticks=[])
ax.imshow(img, cmap='gray')
ax.set_title('PNEUMONIA')

path2 = '../input/chest_xray/chest_xray/train/NORMAL'
name2 = os.listdir(path2)[1]
img2 = Image.open(path2+"/"+name2)
ax = fig.add_subplot(1, 5,  2, xticks=[], yticks=[])
ax.imshow(img2, cmap='gray')
ax.set_title('NORMAL')

In [ ]:
classes = ['PNEUMONIA', 'NORMAL']

In [ ]:
!wget https://raw.githubusercontent.com/Iamsdt/60daysofudacity/master/day22/Helper.py

In [ ]:
import Helper
import torch
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader

data_dir = '../input/chest_xray/chest_xray'

mean = [0.485, 0.450, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
                                transforms.Resize(255),
                                transforms.CenterCrop(224),
                                #transforms.RandomResizedCrop(224),
                                #transforms.RandomHorizontalFlip(),
                                #transforms.ColorJitter(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])

test_transform = transforms.Compose([
                                transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])


train_data = datasets.ImageFolder(data_dir+"/train", transform=train_transform)
test_data = datasets.ImageFolder(data_dir+"/test", transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

print(len(train_loader))
print(len(test_loader))

# Visualize

In [ ]:
Helper.visualize(train_loader, classes)

# Load Model

In [ ]:
model = models.vgg16(pretrained=True)
model.classifier

In [ ]:
model = Helper.freeze_parameters(model)

In [ ]:
import torch.nn as nn
from collections import OrderedDict

classifier = nn.Sequential(
  nn.Linear(in_features=25088, out_features=4096),
  nn.ReLU(inplace=True),
  nn.Dropout(p=0.5),
  nn.Linear(in_features=4096, out_features=4096),
  nn.ReLU(inplace=True),
  nn.Dropout(p=0.4),
  nn.Linear(in_features=4096, out_features=2),
  nn.LogSoftmax(dim=1)
)
    
model.classifier = classifier
model.classifier

In [ ]:
import torch.optim as optim
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.ASGD(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.01)

# Train

Due to lower accuracy model training is stop now

In [ ]:
epoch = 5

In [ ]:
model, train_loss, test_loss = Helper.train(model, train_loader, test_loader, epoch, optimizer, criterion, scheduler)

In [ ]:
# model = Helper.load_latest_model(model)

In [ ]:
Helper.check_overfitted(train_loss, test_loss)

# Test

In [ ]:
# Helper.test_per_class(model, test_loader, criterion, classes)

In [ ]:
# Helper.test(model, test_loader, criterion)

# Single Image

In [ ]:
from PIL import Image

transform = transforms.Compose([
                                transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])

def test(file):
    ids = train_loader.dataset.class_to_idx
    file = Image.open(file).convert('RGB') 
    img = transform(file).unsqueeze(0)
    with torch.no_grad():
        out = model(img.to(device))
        ps = torch.exp(out)
        top_p, top_class = ps.topk(1, dim=1)
        value = top_class.item()
        print("Value:", value)
        print(classes[value])
        plt.imshow(np.array(file))
        plt.show()

# PNEUMONIA

In [ ]:
from matplotlib import pyplot as plt
path = data_dir+"/val/PNEUMONIA"
path = path+"/"+os.listdir(path)[0]
path
test(path)

In [ ]:
path = data_dir+"/val/PNEUMONIA"
path = path+"/"+os.listdir(path)[5]
path
test(path)

In [ ]:
path = data_dir+"/test/PNEUMONIA"
path = path+"/"+os.listdir(path)[0]
path
test(path)

# NORMAL

In [ ]:
path = data_dir+"/val/NORMAL"
path = path+"/"+os.listdir(path)[0]
path
test(path)

In [ ]:
path = data_dir+"/val/NORMAL"
path = path+"/"+os.listdir(path)[5]
path
test(path)

In [ ]:
path = data_dir+"/test/NORMAL"
path = path+"/"+os.listdir(path)[0]
path
test(path)

# Test with new images

In [ ]:
valid_transform = transforms.Compose([
                                transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean, std)])

valid_data = datasets.ImageFolder(data_dir+"/val", transform=valid_transform)

validloader = torch.utils.data.DataLoader(valid_data, batch_size=10)
len(validloader)

In [ ]:
# Helper.test_per_class(model, test_loader, criterion, classes)

In [ ]:
# Helper.test(model, test_loader, criterion)